# Data Preprocessing
We want to build a model that predicts whether or not someone will respond to an offer. Many variables need to be changed to be numeric types for a machine learning model to process it. We also need to combine our data. Please see the steps required below for each of our datasets.

In [2]:
import pandas as pd
import numpy as np
import math
import json
# % matplotlib inline
import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

## Portfolio

In [3]:
portfolio.dtypes

reward         int64
channels      object
difficulty     int64
duration       int64
offer_type    object
id            object
dtype: object

In [4]:
portfolio.head(5)

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [5]:
# machine learning algorithms can only handle numerical features, change the channels column to dummy variables
# use offer_type column to create dummy variables
portfolio = pd.concat([portfolio, pd.get_dummies(portfolio['offer_type'].apply(pd.Series).stack()).sum(level=0)], axis=1)
# drop the offer_type column
portfolio.drop('offer_type', axis=1, inplace=True)
# use channel column to create dummy variables
portfolio = pd.concat([portfolio, pd.get_dummies(portfolio['channels'].apply(pd.Series).stack()).sum(level=0)], axis=1)
# drop channel column
portfolio.drop('channels', axis=1, inplace=True)

In [6]:
portfolio= portfolio.rename(columns={'id':'offer_id'})

In [7]:
portfolio.head()

,reward,difficulty,duration,offer_id,bogo,discount,informational,email,mobile,social,web
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,1,1,1,0
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0,1
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0,1
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0,1


## Profile

In [8]:
profile.dtypes

gender               object
age                   int64
id                   object
became_member_on      int64
income              float64
dtype: object

In [9]:
# change the became_member_on column to datetime
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')

In [10]:
# machine learning algorithms cannot consume dates
# change became_member_on to number of days as a member 
profile['days_as_member'] = (datetime.datetime.today() - profile['became_member_on']).dt.days

# drop became_member_on column
profile.drop('became_member_on', axis=1, inplace=True)

In [11]:
# filter for all the customers who have income data and no gender data
profile[profile['gender'].isnull() & profile['income'].notnull()]

,gender,age,id,income,days_as_member


In [12]:
profile[profile['income'].isnull() & profile['gender'].notnull()]

,gender,age,id,income,days_as_member


NOTE: because there are only 4 features and all the rows with missing genders also have missing income values, we can drop these rows

In [13]:
profile[profile['age'] >= 118]

,gender,age,id,income,days_as_member
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,NaN,2325
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,NaN,1810
4,None,118,a03223e636434f42ac4c3df47e8bac43,NaN,2152
6,None,118,8ec6ce2a7e7949b1bf142def7d0e0586,NaN,2100
7,None,118,68617ca6246f4fbc85e91a2a49552598,NaN,2093
...,...,...,...,...,...
16980,None,118,5c686d09ca4d475a8f750f2ba07e0440,NaN,2489
16982,None,118,d9ca82f550ac4ee58b6299cf1e5c824a,NaN,2628
16989,None,118,ca45ee1883624304bac1e4c8a114f045,NaN,1939
16991,None,118,a9a20fa8b5504360beb4e7c8712f8306,NaN,2718


NOTE: it is also the same records with invalid age values, we will drop these columns.

In [14]:
profile.shape

(17000, 5)

In [15]:
# drop nan values
profile.dropna(inplace=True)

In [16]:
profile.shape

(14825, 5)

In [17]:
print(f"{17000-14825} records dropped")

2175 records dropped


In [18]:
profile[profile['income'].isnull()]

,gender,age,id,income,days_as_member


In [19]:
profile.head()

,gender,age,id,income,days_as_member
1,F,55,0610b486422d4921ae7d2bf64640c50b,112000.0,2172
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,2239
5,M,68,e2127556f4f64592b11af22de27a7932,70000.0,1887
8,M,65,389bc3fa690240e798340f5a15918d5c,53000.0,1963
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,2053


In [20]:
profile.gender.value_counts()

M    8484
F    6129
O     212
Name: gender, dtype: int64

In [21]:
# machine learning algorithms can only handle numerical features, change the gender column to dummy variables
# create gender dummy variables
profile = pd.concat([profile, pd.get_dummies(profile['gender'].apply(pd.Series).stack()).sum(level=0)], axis=1)

In [22]:
# drop channel column
profile.drop('gender', axis=1, inplace=True)

In [23]:
profile.head()

,age,id,income,days_as_member,F,M,O
1,55,0610b486422d4921ae7d2bf64640c50b,112000.0,2172,1,0,0
3,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,2239,1,0,0
5,68,e2127556f4f64592b11af22de27a7932,70000.0,1887,0,1,0
8,65,389bc3fa690240e798340f5a15918d5c,53000.0,1963,0,1,0
12,58,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,2053,0,1,0


In [24]:
profile= profile.rename(columns={'id':'user_id'})

In [25]:
profile.head()

,age,user_id,income,days_as_member,F,M,O
1,55,0610b486422d4921ae7d2bf64640c50b,112000.0,2172,1,0,0
3,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,2239,1,0,0
5,68,e2127556f4f64592b11af22de27a7932,70000.0,1887,0,1,0
8,65,389bc3fa690240e798340f5a15918d5c,53000.0,1963,0,1,0
12,58,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,2053,0,1,0


## Transcript

In [26]:
transcript.dtypes

person    object
event     object
value     object
time       int64
dtype: object

In [27]:
transcript.tail()

,person,event,value,time
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714
306533,c02b10e8752c4d8e9b73f918558531f7,transaction,{'amount': 4.05},714


In [28]:
transcript.sample(n=5)

,person,event,value,time
174575,e0af91a9e5b041f59c6ff8f6df50bcd7,transaction,{'amount': 24.26},426
177472,4d5c56fde0e440e1bb838a86a26726fe,transaction,{'amount': 24.36},432
207148,b85acef400ab40cca023cb5895e07d97,offer received,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},504
134264,376ca2bef2a64d12af12d1ce52423eed,offer viewed,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},354
17719,ed213022de554001874b97fcff479283,offer viewed,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},6


In [29]:
# convert the json column to usable columns

In [30]:
# transcript['value_type'] = transcript['value'].apply(lambda x: list(x.keys())[0])
# transcript['value_details'] = transcript['value'].apply(lambda x: list(x.values())[0])
# transcript['value_details'] = transcript['value_details'].astype(str)
# this doesnt work for joining later

In [31]:
# unpack the value column
transcript['offer_id'] = transcript['value'].apply(lambda x: x.get('offer_id') or x.get('offer id'))
transcript['amount'] = transcript['value'].apply(lambda x: x.get('amount'))

In [32]:
# drop the value column
transcript.drop(columns=['value'], inplace=True)

In [33]:
# drop all the duplicate records
transcript.drop_duplicates(inplace=True)

## Combining data

In [34]:
portfolio.head(2)

,reward,difficulty,duration,offer_id,bogo,discount,informational,email,mobile,social,web
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,1,1,1,0
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1,1


In [35]:
profile.head(2)

,age,user_id,income,days_as_member,F,M,O
1,55,0610b486422d4921ae7d2bf64640c50b,112000.0,2172,1,0,0
3,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,2239,1,0,0


In [36]:
transcript.sample(n=5)

,person,event,time,offer_id,amount
200693,af5215439a5c4c6f9a0955b609de4851,offer completed,498,2298d6c36e964ae4a3e7e9706d1fb8c2,NaN
150454,d3cddb41ae3b43edaa67777501c32e45,transaction,402,None,21.04
230441,9df2790f060d430e94f17189042e5671,offer completed,534,fafdcd668e3743c1bb461111dcafc2a4,NaN
18955,d15e46c18322472ab7e3dfbc2951aefa,transaction,12,None,8.36
201678,fe97aa22dd3e48c8b143116a8403dd52,offer received,504,3f207df678b143eea3cee63160fa8bed,NaN


In [37]:
transcript.shape

(306137, 5)

In [38]:
# combine datasets on corresponding ids
combined = pd.merge(transcript, profile, left_on= 'person', right_on='user_id', how='left')
combined = pd.merge(combined, portfolio, on = 'offer_id', how='left')
combined.sample(n=5)

,person,event,time,offer_id,amount,age,user_id,income,days_as_member,F,...,reward,difficulty,duration,bogo,discount,informational,email,mobile,social,web
197585,6bbba83ef9204264a0ad9e9a73153886,transaction,486,None,6.95,84.0,6bbba83ef9204264a0ad9e9a73153886,61000.0,2253.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93611,86473e1e0145431d8c7d7b0d39a86fd4,transaction,240,None,0.30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115698,259049daa72f43b29bba5758db435f9d,offer received,336,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,5.0,7.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
28091,9299c46942f84bb6b5e98fd78fbb9d41,offer viewed,42,fafdcd668e3743c1bb461111dcafc2a4,NaN,70.0,9299c46942f84bb6b5e98fd78fbb9d41,88000.0,1943.0,0.0,...,2.0,10.0,10.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
203475,77c449768ead487b963f3c4171f2ad86,offer received,504,5a8bc65990b245e5a138643cd4eb9837,NaN,28.0,77c449768ead487b963f3c4171f2ad86,46000.0,2007.0,0.0,...,0.0,0.0,3.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0


In [39]:
# no records gained, correct
combined.shape

(306137, 22)

In [40]:
# find all the nan values
combined.isnull().sum()

person                 0
event                  0
time                   0
offer_id          138953
amount            167184
age                33749
user_id            33749
income             33749
days_as_member     33749
F                  33749
M                  33749
O                  33749
reward            138953
difficulty        138953
duration          138953
bogo              138953
discount          138953
informational     138953
email             138953
mobile            138953
social            138953
web               138953
dtype: int64

In [41]:
# there are no users associated with these records
combined[combined['user_id'].isnull()].sample(n=15)

,person,event,time,offer_id,amount,age,user_id,income,days_as_member,F,...,reward,difficulty,duration,bogo,discount,informational,email,mobile,social,web
55686,c4196572916f4e7a8a5e6b0c18c7dd40,offer received,168,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,3.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
48569,06b1031271174d8596c1996478f07ede,transaction,138,None,2.38,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184382,235c1e2f7a4c4251bb4333a09a9c523b,offer viewed,444,ae264e3637204a6fb9bb56bc8210ddfd,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,10.0,7.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
224174,875d632bed8e42a4acf7e8e981781750,offer viewed,516,2298d6c36e964ae4a3e7e9706d1fb8c2,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,7.0,7.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
191302,212a2120b14d4ed88a10d2524c715b28,transaction,468,None,0.36,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73735,8b3e8a3f563347e49c139b4469889c62,offer viewed,180,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,10.0,5.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
53232,a89752cdfdf24779aa22faa2bc24e7c2,offer received,168,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,5.0,5.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
265917,169a67c164814dabad8bb7c54fb1cc77,transaction,588,None,1.23,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250820,264fdf78a4f240bd981b122ee31a9e2d,offer received,576,ae264e3637204a6fb9bb56bc8210ddfd,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,10.0,7.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
210335,3a224439e19e4c2096262bab837a3773,offer received,504,ae264e3637204a6fb9bb56bc8210ddfd,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,10.0,7.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0


In [42]:
profile[profile.user_id == '5ae36f912be1492199ec2da838cc6dda']

,age,user_id,income,days_as_member,F,M,O


In [43]:
# since we are interested in how users respond to offers, we can drop all the records where there is no user associated
combined.dropna(subset=['user_id'], inplace=True)

In [44]:
# all transaction records dont correlate with any offers necessarily (no direct link) do we need to keep them?
combined[combined['reward'].isnull()].sample(n=15)

,person,event,time,offer_id,amount,age,user_id,income,days_as_member,F,...,reward,difficulty,duration,bogo,discount,informational,email,mobile,social,web
36016,f08b99e118ee48c2b6abb3b46c6f19d4,transaction,72,None,11.41,46.0,f08b99e118ee48c2b6abb3b46c6f19d4,52000.0,2257.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46730,4c48d368ead5483e99b1b97c5e4d4481,transaction,126,None,11.47,89.0,4c48d368ead5483e99b1b97c5e4d4481,93000.0,1858.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240352,4fd8b68d55fb4469a3cfb235b078063c,transaction,558,None,20.69,56.0,4fd8b68d55fb4469a3cfb235b078063c,72000.0,2777.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174304,ebe7ef46ea6f4963a7dd49f501b26779,transaction,426,None,0.69,59.0,ebe7ef46ea6f4963a7dd49f501b26779,41000.0,3078.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96454,b1b341e9be7b4c3380451a8071957029,transaction,258,None,10.09,22.0,b1b341e9be7b4c3380451a8071957029,65000.0,2617.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277438,31a99f67ed9f4b859c4279e55e11d3fb,transaction,612,None,1.15,43.0,31a99f67ed9f4b859c4279e55e11d3fb,39000.0,1977.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280364,91ccf5dc4998403e94205f194efd95e0,transaction,618,None,14.56,40.0,91ccf5dc4998403e94205f194efd95e0,79000.0,2851.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227139,2430f060c40a4ab9915b763d075b36ab,transaction,522,None,8.84,54.0,2430f060c40a4ab9915b763d075b36ab,72000.0,1895.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191476,e70fc853136a4632beffc2335f571dfc,transaction,468,None,19.31,70.0,e70fc853136a4632beffc2335f571dfc,97000.0,2457.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264523,a5db8bfaf48043e988949acadbc5b18e,transaction,582,None,31.89,79.0,a5db8bfaf48043e988949acadbc5b18e,96000.0,1990.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# find all the values where amount is null
combined[combined['amount'].isnull()].event.value_counts()
# this is expected as amount null for offer records

offer received     66501
offer viewed       49860
offer completed    32070
Name: event, dtype: int64

In [46]:
# find all the values where offer_id is null
combined[combined['offer_id'].isnull()].event.value_counts()
# this is expected as offer_id null for transaction records

transaction    123957
Name: event, dtype: int64

### Split transaction and offer records for analysis


In [47]:
# find all the nan values
combined.isnull().sum()

person                 0
event                  0
time                   0
offer_id          123957
amount            148431
age                    0
user_id                0
income                 0
days_as_member         0
F                      0
M                      0
O                      0
reward            123957
difficulty        123957
duration          123957
bogo              123957
discount          123957
informational     123957
email             123957
mobile            123957
social            123957
web               123957
dtype: int64

In [48]:
# split the data into offer and transaction records
offer_df = combined[combined['offer_id'].notnull()]
transaction_df = combined[combined['offer_id'].isnull()]

In [49]:
print(combined.shape)
print(offer_df.shape)
print(transaction_df.shape)

(272388, 22)
(148431, 22)
(123957, 22)


#### Transaction records

In [50]:
transaction_df.sample(n=5)

,person,event,time,offer_id,amount,age,user_id,income,days_as_member,F,...,reward,difficulty,duration,bogo,discount,informational,email,mobile,social,web
227753,532638c9f8d942a5bb39688d2f8bac20,transaction,528,None,24.13,80.0,532638c9f8d942a5bb39688d2f8bac20,87000.0,2161.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243780,e728ea2be7844d7eb0da24f4e1444c44,transaction,570,None,21.53,64.0,e728ea2be7844d7eb0da24f4e1444c44,73000.0,2052.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32552,f63428edea1e408bb92692e7f02f4e02,transaction,54,None,22.81,68.0,f63428edea1e408bb92692e7f02f4e02,97000.0,3095.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191654,c91761144bce489bac76377c73b4058a,transaction,468,None,2.00,54.0,c91761144bce489bac76377c73b4058a,48000.0,2972.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89448,435e8d41374d4c9ca365a90566f28e0a,transaction,228,None,27.13,54.0,435e8d41374d4c9ca365a90566f28e0a,77000.0,1850.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# drop all the id columns as we do not need them for analysis
transaction_df.drop(columns=['person', 'offer_id'], inplace=True)

/tmp/ipykernel_22775/2794338413.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_df.drop(columns=['person', 'offer_id'], inplace=True)


In [52]:
# drop all the offer related columns not related to transactions (nan values)
transaction_df.drop(columns=['reward', 'difficulty', 'duration', 'email', 'mobile', 'social', 'web'], inplace=True)

/tmp/ipykernel_22775/2144275890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_df.drop(columns=['reward', 'difficulty', 'duration', 'email', 'mobile', 'social', 'web'], inplace=True)


In [53]:
transaction_df.head()

,event,time,amount,age,user_id,income,days_as_member,F,M,O,bogo,discount,informational
12654,transaction,0,0.83,20.0,02c083884c7d45b39cc68e1314fec56c,30000.0,2541.0,1.0,0.0,0.0,NaN,NaN,NaN
12657,transaction,0,34.56,42.0,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,96000.0,2717.0,0.0,1.0,0.0,NaN,NaN,NaN
12659,transaction,0,13.23,36.0,54890f68699049c2a04d415abc25e717,56000.0,2006.0,0.0,1.0,0.0,NaN,NaN,NaN
12670,transaction,0,19.51,55.0,b2f1cd155b864803ad8334cdf13c4bd2,94000.0,2079.0,1.0,0.0,0.0,NaN,NaN,NaN
12671,transaction,0,18.97,39.0,fe97aa22dd3e48c8b143116a8403dd52,67000.0,2017.0,1.0,0.0,0.0,NaN,NaN,NaN


#### Offer records

In [54]:
offer_df.sample(n=5)

,person,event,time,offer_id,amount,age,user_id,income,days_as_member,F,...,reward,difficulty,duration,bogo,discount,informational,email,mobile,social,web
129775,f992e2569e2744fbb92e824fac69d2e4,offer viewed,342,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,78.0,f992e2569e2744fbb92e824fac69d2e4,52000.0,2265.0,0.0,...,10.0,10.0,5.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
213807,471b265931d34361aee6f5b7a62f9a94,offer received,504,3f207df678b143eea3cee63160fa8bed,NaN,39.0,471b265931d34361aee6f5b7a62f9a94,45000.0,2170.0,1.0,...,0.0,0.0,4.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
198050,a4c5415b4a7648d185dfd0379dde947a,offer viewed,486,2906b810c7d4411798c6938adc9daaa5,NaN,62.0,a4c5415b4a7648d185dfd0379dde947a,87000.0,2139.0,1.0,...,2.0,10.0,7.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
17585,b1975494797c4707840949d311ca2574,offer viewed,6,3f207df678b143eea3cee63160fa8bed,NaN,71.0,b1975494797c4707840949d311ca2574,54000.0,1802.0,0.0,...,0.0,0.0,4.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
257177,c94c15ddd30845faaf072d20b111aa1e,offer received,576,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,67.0,c94c15ddd30845faaf072d20b111aa1e,54000.0,2483.0,1.0,...,5.0,5.0,7.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0


In [55]:
# drop all the transactions related columns not related to offers (nan values)
offer_df.drop(columns=['amount'], inplace=True)
offer_df.drop(columns=['person'], inplace=True)

# the time column is no of interesting to us, we can drop it
# offer_df.drop(columns=['time'], inplace=True)

/tmp/ipykernel_22775/3663222115.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offer_df.drop(columns=['amount'], inplace=True)
/tmp/ipykernel_22775/3663222115.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offer_df.drop(columns=['person'], inplace=True)


In [56]:
portfolio

,reward,difficulty,duration,offer_id,bogo,discount,informational,email,mobile,social,web
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,1,1,1,0
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0,1
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0,1
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0,1
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,1,1,1,0
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,0,1


In [57]:
offer_df[offer_df['informational'] == 1].event.value_counts()

offer received    13300
offer viewed       9360
Name: event, dtype: int64

In [58]:
# informational offers are never completed, so we can drop them 
offer_df = offer_df[offer_df['informational'] != 1]

In [59]:
offer_df.sample(n=5)

,event,time,offer_id,age,user_id,income,days_as_member,F,M,O,reward,difficulty,duration,bogo,discount,informational,email,mobile,social,web
207445,offer received,504,f19421c1d4aa40978ebb69ca19b0e20d,47.0,2e24567caebc4fecb582efc09a26403c,67000.0,3068.0,1.0,0.0,0.0,5.0,5.0,5.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
37800,offer completed,78,fafdcd668e3743c1bb461111dcafc2a4,52.0,62c99873e4364232955db9f0ebaf042c,101000.0,2425.0,1.0,0.0,0.0,2.0,10.0,10.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
251857,offer received,576,4d5c57ea9a6940dd891ad53e9dbe8da0,92.0,2b45d1b9f77440538af83122e3d14a6c,65000.0,2662.0,1.0,0.0,0.0,10.0,10.0,5.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
211751,offer received,504,0b1e1539f2cc45b7b9fa7c272da2e1d7,74.0,9c860622473e4c6c8e57b6caa7523843,71000.0,3421.0,0.0,1.0,0.0,5.0,20.0,10.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
245696,offer received,576,ae264e3637204a6fb9bb56bc8210ddfd,64.0,162e7d9c644b438ea4606e337df0d3d1,60000.0,2624.0,0.0,1.0,0.0,10.0,10.0,7.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0


In [60]:
offer_df[(offer_df.user_id == 'bea062a97557458a97f3e2df8d87755a') & (offer_df.offer_id == '4d5c57ea9a6940dd891ad53e9dbe8da0')]

,event,time,offer_id,age,user_id,income,days_as_member,F,M,O,reward,difficulty,duration,bogo,discount,informational,email,mobile,social,web
206535,offer received,504,4d5c57ea9a6940dd891ad53e9dbe8da0,69.0,bea062a97557458a97f3e2df8d87755a,99000.0,2328.0,1.0,0.0,0.0,10.0,10.0,5.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
215894,offer viewed,504,4d5c57ea9a6940dd891ad53e9dbe8da0,69.0,bea062a97557458a97f3e2df8d87755a,99000.0,2328.0,1.0,0.0,0.0,10.0,10.0,5.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
231216,offer completed,534,4d5c57ea9a6940dd891ad53e9dbe8da0,69.0,bea062a97557458a97f3e2df8d87755a,99000.0,2328.0,1.0,0.0,0.0,10.0,10.0,5.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0


In [61]:
# use the event column to create dummy variables
offer_complete_df = pd.concat([offer_df, pd.get_dummies(offer_df['event'].apply(pd.Series).stack()).sum(level=0)], axis=1)
# rename offer completed column to offer_completed, offer received to offer_received, offer viewed to offer_viewed
offer_complete_df.rename(columns={'offer completed':'offer_completed', 'offer received':'offer_received', 'offer viewed':'offer_viewed'}, inplace=True)


In [62]:
offer_complete_df[(offer_complete_df.user_id == 'bea062a97557458a97f3e2df8d87755a') & (offer_complete_df.offer_id == '4d5c57ea9a6940dd891ad53e9dbe8da0')]

,event,time,offer_id,age,user_id,income,days_as_member,F,M,O,...,bogo,discount,informational,email,mobile,social,web,offer_completed,offer_received,offer_viewed
206535,offer received,504,4d5c57ea9a6940dd891ad53e9dbe8da0,69.0,bea062a97557458a97f3e2df8d87755a,99000.0,2328.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0,1,0
215894,offer viewed,504,4d5c57ea9a6940dd891ad53e9dbe8da0,69.0,bea062a97557458a97f3e2df8d87755a,99000.0,2328.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0,0,1
231216,offer completed,534,4d5c57ea9a6940dd891ad53e9dbe8da0,69.0,bea062a97557458a97f3e2df8d87755a,99000.0,2328.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1,0,0


In [63]:
# create a combined id to group the data by
# we will use these groups and logic to see if an offer that was viewed was also completed
offer_complete_df['combined_id'] = offer_complete_df.apply(lambda x: x['user_id'] + x['offer_id'], axis=1)
combined_id_map = offer_complete_df.groupby('combined_id')

In [64]:
def offer_viewed_and_completed( row, combined_id_map):

    if row['event'] != 'offer viewed':
        return 0
    
    combined_id = row['user_id'] + row['offer_id']
    filtered_rows = combined_id_map.get_group(combined_id)

    # find the previous offer received event
    previous_offer_received = filtered_rows[(filtered_rows['time'] <= row['time']) & (filtered_rows['event'] == 'offer received')].tail(1)
    # find the next offer completed event
    next_offer_completed = filtered_rows[(filtered_rows['time'] >= row['time']) & (filtered_rows['event'] == 'offer completed')].head(1)

    if previous_offer_received.shape[0] == 0 or next_offer_completed.shape[0] == 0:
        return 0

    if (next_offer_completed['time'].iloc[0] - previous_offer_received['time'].iloc[0] < 24 * row['duration']):
        return 1
    
    return 0

offer_complete_df['offer_viewed_and_completed'] = offer_complete_df.apply(lambda x: offer_viewed_and_completed(x, combined_id_map), axis=1)

In [65]:
# test the logic
offer_complete_df[(offer_complete_df.user_id == '0020c2b971eb4e9188eac86d93036a77') & (offer_complete_df.offer_id == 'fafdcd668e3743c1bb461111dcafc2a4')][['event', 'offer_completed','offer_received', 'offer_viewed', 'combined_id','offer_viewed_and_completed']]

,event,offer_completed,offer_received,offer_viewed,combined_id,offer_viewed_and_completed
1889,offer received,0,1,0,0020c2b971eb4e9188eac86d93036a77fafdcd668e3743...,0
18431,offer viewed,0,0,1,0020c2b971eb4e9188eac86d93036a77fafdcd668e3743...,1
31327,offer completed,1,0,0,0020c2b971eb4e9188eac86d93036a77fafdcd668e3743...,0
112684,offer received,0,1,0,0020c2b971eb4e9188eac86d93036a77fafdcd668e3743...,0
218771,offer completed,1,0,0,0020c2b971eb4e9188eac86d93036a77fafdcd668e3743...,0


In [66]:
# test the logic
offer_complete_df[(offer_complete_df.user_id == '018a49ffb8cf4812903e7c1f56fbb0b0') & (offer_complete_df.offer_id == 'f19421c1d4aa40978ebb69ca19b0e20d')][['event', 'offer_completed','offer_received', 'offer_viewed', 'combined_id','offer_viewed_and_completed']]

,event,offer_completed,offer_received,offer_viewed,combined_id,offer_viewed_and_completed
4254,offer received,0,1,0,018a49ffb8cf4812903e7c1f56fbb0b0f19421c1d4aa40...,0
13653,offer viewed,0,0,1,018a49ffb8cf4812903e7c1f56fbb0b0f19421c1d4aa40...,1
34599,offer completed,1,0,0,018a49ffb8cf4812903e7c1f56fbb0b0f19421c1d4aa40...,0
205689,offer received,0,1,0,018a49ffb8cf4812903e7c1f56fbb0b0f19421c1d4aa40...,0
228530,offer viewed,0,0,1,018a49ffb8cf4812903e7c1f56fbb0b0f19421c1d4aa40...,1
239790,offer completed,1,0,0,018a49ffb8cf4812903e7c1f56fbb0b0f19421c1d4aa40...,0
249162,offer received,0,1,0,018a49ffb8cf4812903e7c1f56fbb0b0f19421c1d4aa40...,0
266409,offer viewed,0,0,1,018a49ffb8cf4812903e7c1f56fbb0b0f19421c1d4aa40...,1
269576,offer completed,1,0,0,018a49ffb8cf4812903e7c1f56fbb0b0f19421c1d4aa40...,0


In [67]:
# the logic works as expected

In [68]:
offer_complete_df.sample(n= 5)

,event,time,offer_id,age,user_id,income,days_as_member,F,M,O,...,informational,email,mobile,social,web,offer_completed,offer_received,offer_viewed,combined_id,offer_viewed_and_completed
176981,offer viewed,426,fafdcd668e3743c1bb461111dcafc2a4,57.0,a38d0193a34a45dabbc0daab567cb670,80000.0,2004.0,0.0,1.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0,0,1,a38d0193a34a45dabbc0daab567cb670fafdcd668e3743...,0
136326,offer viewed,360,2298d6c36e964ae4a3e7e9706d1fb8c2,60.0,a7dc060f6fc94ca7bf71fbb188187dca,69000.0,2336.0,0.0,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,0,0,1,a7dc060f6fc94ca7bf71fbb188187dca2298d6c36e964a...,1
55573,offer received,168,9b98b8c7a33c4b65b9aebfe6a799e6d9,79.0,f9c0574976aa495cbd957ea79cabc991,66000.0,3184.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0,1,0,f9c0574976aa495cbd957ea79cabc9919b98b8c7a33c4b...,0
177922,offer viewed,432,ae264e3637204a6fb9bb56bc8210ddfd,36.0,96972ee997924865b4c939de10088622,53000.0,1933.0,0.0,1.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0,0,1,96972ee997924865b4c939de10088622ae264e3637204a...,0
7124,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,46.0,872683bcd7534cebbed03becb39014d9,96000.0,3225.0,1.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0,1,0,872683bcd7534cebbed03becb39014d9fafdcd668e3743...,0


In [69]:
# as we are only interested in offers that were viewed and then completed, we can drop all the non viewed records
offer_complete_df = offer_complete_df[offer_complete_df['offer_viewed'] != 0]

In [70]:
# drop any further columns that will not be use for modeling
offer_complete_df.drop(columns=['event','offer_received','time','offer_id','user_id','combined_id','informational', 'offer_completed', 'offer_viewed'], inplace=True)

/tmp/ipykernel_22775/3113366186.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offer_complete_df.drop(columns=['event','offer_received','time','offer_id','user_id','combined_id','informational', 'offer_completed', 'offer_viewed'], inplace=True)


In [71]:
offer_complete_df.sample(n= 5)

,age,income,days_as_member,F,M,O,reward,difficulty,duration,bogo,discount,email,mobile,social,web,offer_viewed_and_completed
26392,63.0,65000.0,2038.0,1.0,0.0,0.0,5.0,20.0,10.0,0.0,1.0,1.0,0.0,0.0,1.0,1
178484,84.0,37000.0,2039.0,0.0,1.0,0.0,10.0,10.0,5.0,1.0,0.0,1.0,1.0,1.0,1.0,0
82196,75.0,63000.0,2048.0,1.0,0.0,0.0,10.0,10.0,7.0,1.0,0.0,1.0,1.0,1.0,0.0,1
35654,59.0,63000.0,1926.0,0.0,1.0,0.0,2.0,10.0,10.0,0.0,1.0,1.0,1.0,1.0,1.0,0
170878,26.0,41000.0,2097.0,1.0,0.0,0.0,10.0,10.0,5.0,1.0,0.0,1.0,1.0,1.0,1.0,0


#### Write data to parquet files

In [72]:
# write output data to parquet files
# combined.to_parquet('data/combined.parquet', engine='pyarrow')
# offer_df.to_parquet('data/offer_df.parquet', engine='pyarrow')
offer_complete_df.to_parquet('data/offer_complete_df.parquet', engine='pyarrow')
transaction_df.to_parquet('data/transaction_df.parquet', engine='pyarrow')